### Imports

In [1]:
%matplotlib inline

In [2]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import pandas as pd

import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
# import tkinter


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

### Data formatting en reading

In [3]:
SOS_token = 0 # Start of sentence
EOS_token = 1 # End of sentence

In [4]:
class Language:
    def __init__(self):
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: 'SOS', 1: 'EOS'}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

    def readLang(self, s):
        for index, value in s.items():
            value = normalizeString(value)
            self.addSentence(value)

    def showLang(self):
        print("-- NL Language")
        print("Word count: ", sum(self.word2count.values()))
        print("Vocabe size: ", self.n_words)

In [5]:
class MRLanguage:
    def __init__(self):
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: 'SOS', 1: 'EOS'}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(', '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

    def readLang(self, s):
        for index, value in s.items():
            value = normalizeMRString(value)
            self.addSentence(value)

    def showLang(self):
        print("-- MR Language")
        print("Word count: ", sum(self.word2count.values()))
        print("Vocabe size: ", self.n_words)

In [6]:
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?_]+", r" ", s)
    return s

def normalizeMRString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r'\[[^]]*\]', r"", s) # TODO: Data in de square brackets is lost!!
    # s = re.sub(r"[\[*\]]+", r" ", s)
    return s

In [7]:
def indexesFromSentence(lang, sentence, delimeter=' '):
    indices = [lang.word2index[word] for word in sentence.split(delimeter)]
    return torch.tensor(indices)

def sentenceFromIndexes(lang, indexes, delimeter=' '):
    sentence = ''
    for i in indexes:
        sentence += str(lang.index2word[i.item()]) + delimeter
    
    return sentence


def get_output_data(file):
    corpus_df = pd.read_csv(file)
    lang_df = corpus_df['ref']

    natural_lang = Language()
    natural_lang.readLang(lang_df)
    # corpus_df['ref'] = corpus_df['ref'].astype(str) + ' eos'
    corpus_df['ref'] = corpus_df['ref'].apply(lambda x: normalizeString(x))
    corpus_df['ref'] = corpus_df['ref'].apply(lambda x: indexesFromSentence(natural_lang, x))

    return corpus_df['ref'], natural_lang

def get_input_data(file):
    corpus_df = pd.read_csv(file)
    lang_df = corpus_df['mr']

    mr_lang = MRLanguage()
    mr_lang.readLang(corpus_df['mr'])
    # corpus_df['mr'] = corpus_df['mr'].astype(str) + ', eos'
    corpus_df['mr'] = corpus_df['mr'].apply(lambda x: normalizeMRString(x))
    corpus_df['mr'] = corpus_df['mr'].apply(lambda x: indexesFromSentence(mr_lang, x, delimeter=", "))

    return corpus_df['mr'], mr_lang

Actually reading the data

In [8]:
# GET DATA
print("Reading...")
path = "delexicalized/delex_only.csv"
input_data, mr_lang = get_input_data(path) #TODO: Correctly read all info of labels
target_data, nl_lang = get_output_data(path)

print(mr_lang.showLang())
print(nl_lang.showLang())

Reading...
-- MR Language
Word count:  10632
Vocabe size:  10
None
-- NL Language
Word count:  32610
Vocabe size:  671
None


### Models

In [9]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)


In [10]:
class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=75):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)

        attn_weights = F.softmax(
            self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))

        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        output = F.relu(output)
        output, hidden = self.gru(output, hidden)

        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

### Training

In [11]:
teacher_forcing_ratio = 0.5


def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=75):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0

    for ei in range(input_length):   
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]], device=device)

    decoder_hidden = encoder_hidden
    
    for di in range(target_length):
        decoder_output, decoder_hidden, decoder_attention = decoder(
            decoder_input, decoder_hidden, encoder_outputs)
        topv, topi = decoder_output.topk(1)
        decoder_input = topi.squeeze().detach()  # detach from history as input
        
        loss += criterion(decoder_output, target_tensor[di])
        if decoder_input.item() == EOS_token:
            break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

In [12]:
import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [21]:
def trainIters(encoder, decoder, n_epoch, learning_rate=0.01):
    print("Started training!")
    
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)

    criterion = nn.NLLLoss()

    for epoch in range(1, n_epoch + 1):
        print(epoch)
        for i in range(len(input_data)):
            input_tensor = input_data[i].view(-1, 1)
            target_tensor = target_data[i].view(-1, 1)
    #         print("Input: ", input_tensor)
    #         print("Target: ", target_tensor)


            loss = train(input_tensor, target_tensor, encoder,
                         decoder, encoder_optimizer, decoder_optimizer, criterion)
            print_loss_total += loss
            plot_loss_total += loss

            
        print_loss_avg = print_loss_total / len(input_data)
        print_loss_total = 0
        print('%s (%d %d%%) %.4f' % (timeSince(start, epoch / n_epoch),
                                     epoch, epoch / n_epoch * 100, print_loss_avg))

        plot_loss_avg = plot_loss_total / len(input_data)
        plot_losses.append(plot_loss_avg)
        plot_loss_total = 0

    showPlot(plot_losses)
    print("Done!")
    return plot_losses

### Plotting results

In [14]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np


def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

### Evaluation

In [15]:
def evaluate(encoder, decoder, input_tensor, max_length=75):
    with torch.no_grad():
#         sentence = sentenceFromIndexes(mr_lang, input_tensor, delimeter=", ")
#         input_tensor = indexesFromSentence(mr_lang, sentence, delimeter=", ")
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []
        decoder_attentions = torch.zeros(max_length, max_length)

        for di in range(max_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            decoder_attentions[di] = decoder_attention.data
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(nl_lang.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words, decoder_attentions[:di + 1]

In [16]:
def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        index = random.randint(0, len(input_data) - 1)
        input_sentence = sentenceFromIndexes(mr_lang, input_data[index], delimeter=", ")
        target_sentence = sentenceFromIndexes(nl_lang, target_data[index], delimeter=" ")
        
        print('>', input_sentence)
        print('=', target_sentence)
        output_words, attentions = evaluate(encoder, decoder, input_data[i])
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

### Running

In [22]:
n_epoch = 100

hidden_size = 256

encoder1 = EncoderRNN(mr_lang.n_words, hidden_size).to(device)
attn_decoder1 = AttnDecoderRNN(hidden_size, nl_lang.n_words, dropout_p=0.1).to(device)

plot_losses = trainIters(encoder1, attn_decoder1, n_epoch)

Started training!
1
2m 15s (- 223m 14s) (1 1%) 3.7979
2
4m 31s (- 221m 53s) (2 2%) 3.5993
3
6m 51s (- 221m 50s) (3 3%) 3.5603
4
9m 13s (- 221m 13s) (4 4%) 3.5366
5
11m 34s (- 219m 57s) (5 5%) 3.5184
6
13m 56s (- 218m 19s) (6 6%) 3.5032
7
16m 17s (- 216m 23s) (7 7%) 3.4886
8
18m 38s (- 214m 27s) (8 8%) 3.4753
9
21m 0s (- 212m 20s) (9 9%) 3.4638
10
23m 21s (- 210m 16s) (10 10%) 3.4496
11
25m 42s (- 208m 3s) (11 11%) 3.4375
12
28m 3s (- 205m 46s) (12 12%) 3.4263
13
30m 26s (- 203m 42s) (13 13%) 3.4132
14
32m 47s (- 201m 25s) (14 14%) 3.4008
15
35m 8s (- 199m 9s) (15 15%) 3.3861
16
37m 29s (- 196m 49s) (16 16%) 3.3709
17
39m 50s (- 194m 31s) (17 17%) 3.3548
18
42m 10s (- 192m 8s) (18 18%) 3.3387
19
44m 31s (- 189m 47s) (19 19%) 3.3246
20
46m 50s (- 187m 22s) (20 20%) 3.3050
21
49m 11s (- 185m 1s) (21 21%) 3.2866
22
51m 31s (- 182m 40s) (22 22%) 3.2672
23
53m 51s (- 180m 19s) (23 23%) 3.2514
24
56m 10s (- 177m 54s) (24 24%) 3.2326
25
58m 30s (- 175m 30s) (25 25%) 3.2114
26
60m 52s (- 173m 1

In [24]:
evaluateRandomly(encoder1, attn_decoder1)

> name, food, pricerange, area, familyfriendly, 
= located food_x city centre is name_x a cheap area_x . 
< name_x eattype_x a eattype_x a near_x near_x near_x near_x . near_x . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .

> name, food, pricerange, customer rating, near, 
= the name_x sells food_x food near near_x . the customer rating is customer rating_x and the price pricerange_x . 
< name_x is a eattype_x eattype_x food_x food food food food food food food food food food food food food food food food food food food food food food food food food food food food food food food food food food food food food food food food food food food food food food food food food food food food food food food food food food food food food food food food food food food food food

> name, food, pricerange, customer rating, familyfriendly, near, 
= for a familyfriendly_x food_x place with pricerange_x places and customer 